In [1]:
# File paths
genes_file = "input-files/tip60-genes.txt"
# Load gene list
with open(genes_file, "r") as f:
    genes = [line.strip() for line in f.readlines()]

illumina_deseq = "DESeq2_results_shrunk.csv" # deseq2 illumina
pacbio_deseq_isoforms = 'DESeq2_results_shrunk_isoforms.csv' # deseq2 pacbio
classifications = 'collapsed_classification.filtered_lite_classification.txt' # pigeon classify
gff = 'gencode.v39.annotation.sorted.gtf' # this will enable cross referencesing the files
p_cutoff = 0.05


## Parse GFF to get lookup tables
gff_lookup = {} # gene_id to gene name
gff_lookup_reverse = {} # gen_name to gene_id
transcript_to_gene = {}
gene_to_transcripts = {}

for line in open(gff):
    if line[0] == '#':
        continue
    elements = line.rstrip().split('\t')
    info = elements[-1].split(';')
    
    if elements[2] == 'gene':
        gene_name = ''
        gene_id = ''
        for thing in info:
            if ' ' not in thing:
                continue
            if thing.split(' ')[1] == 'gene_name':
                gene_name = thing.split(' ')[2].replace('"','')
            if thing.split(' ')[0] == 'gene_id':
                gene_id = thing.split(' ')[1].replace('"','')
        gff_lookup[gene_id] = gene_name
        gff_lookup_reverse[gene_name] = gene_id
    
    if elements[2] == 'transcript':
        transcript_id = ''
        gene_id = ''
        for thing in info:
            if ' ' not in thing:
                continue
            if thing.split(' ')[1] == 'transcript_id':
                transcript_id = thing.split(' ')[2]
            if thing.split(' ')[0] == 'gene_id':
                gene_id = thing.split(' ')[1]

        if gene_id != '' and transcript_id != '':
            transcript_to_gene.setdefault(transcript_id, '')
            transcript_to_gene[transcript_id] = gene_id
            gene_to_transcripts.setdefault(gene_id, set())  # Initialize as a set
            gene_to_transcripts[gene_id].add(transcript_id)


# Parse isoseq classifications
isoform_to_gene = {}
isoform_classification  = {}
for line in open(classifications):
    elements = line.rstrip().split('\t')
    if elements[0] == 'isoform' or elements[0] == '""':
        continue
    isoform  = elements[0].replace('"','')
    ass_gene = elements[6]
    transcript = elements[7]
    category = elements[14]
    
    isoform_classification[isoform] = category
    
    try:
        gene = gff_lookup_reverse[ass_gene]
    except KeyError:
        gene = 'Unknown'
    isoform_to_gene.setdefault(isoform, ['','', ''])
    isoform_to_gene[isoform] = [gene, ass_gene, transcript]
            
            
## parse illumina deseq
illumina_lookup = {}
for line in open(illumina_deseq):
    elements = line.rstrip().split(',')
    gene = elements[0].replace('"','')
    illumina_lookup[gene] = elements

pacbio_lookup = {}
for line in open(pacbio_deseq_isoforms):
    elements=line.rstrip().split(',')
    if elements[1] == 'baseMean' or elements[0] == '""':
        continue
    isoform = elements[0].replace('"','')
    gene = isoform_to_gene[isoform][0]
    
    if gene in gff_lookup.keys():
        gene_name = gff_lookup[gene]
        pacbio_lookup.setdefault(gene_name, [])
        pacbio_lookup[gene_name].append(elements)
    else:
        gene_name = 'Unknown'

In [8]:

print ('Gene\tlog2FoldChange\tpadj\tnum_isoforms\tsignificant_up\tsignificant_down\tcategories')

for cur_gene in genes:
    
    illumina_fold = illumina_lookup[cur_gene][2]
    illumina_p = illumina_lookup[cur_gene][5]
    
    num_isoforms = len(pacbio_lookup[cur_gene])
    categories = []
    num_up = 0
    num_down = 0
    
    for isoform_results in pacbio_lookup[cur_gene]:

        if float(isoform_results[5]) <= 0.05:
            if float(elements[2]) > 0:
                num_up += 1
            else:
                num_down += 1
        isoform = isoform_results[0].replace('"','')
        classification = isoform_classification[isoform]
        categories.append(classification)
    print (cur_gene + '\t' + illumina_fold + '\t' + illumina_p + '\t' + str(num_isoforms) + '\t' + str(num_up) + '\t' + str(num_down) + '\t'  +'|'.join(categories))
        
        

Gene	log2FoldChange	padj	num_isoforms	significant_up	significant_down	categories
KAT5	-0.410763045	0.000225465	13	0	2	reference_match|reference_match|alternative_5end|alternative_5end|alternative_3end|reference_match|combination_of_known_junctions|alternative_3end5end|alternative_5end|3prime_fragment|3prime_fragment|3prime_fragment|3prime_fragment
EP400	-0.280616191	0.038109071	1	0	0	combination_of_known_junctions
TRRAP	-0.138325654	0.414363477	4	0	0	internal_fragment|internal_fragment|internal_fragment|internal_fragment
BRD8	0.157954396	0.35760197	15	0	0	reference_match|combination_of_known_junctions|combination_of_known_junctions|reference_match|combination_of_known_junctions|reference_match|combination_of_known_junctions|combination_of_known_junctions|combination_of_known_junctions|3prime_fragment|3prime_fragment|3prime_fragment|3prime_fragment|reference_match|3prime_fragment
EPC1	0.169092142	0.139968173	6	0	1	alternative_3end|5prime_fragment|5prime_fragment|5prime_fragment|referenc

In [40]:
import pandas as pd

# Define table structure
table_data = []

# Print header
header = ["Gene", "log2FoldChange", "padj", "num_isoforms", "significant_up", "significant_down", "categories"]

for cur_gene in genes:
    illumina_fold = illumina_lookup[cur_gene][2]
    illumina_p = illumina_lookup[cur_gene][5]

    num_isoforms = len(pacbio_lookup[cur_gene])
    categories = []
    num_up = 0
    num_down = 0

    for isoform_results in pacbio_lookup[cur_gene]:
        if float(isoform_results[5]) <= 0.05: # noramlly 5 for most tables
            if float(isoform_results[2]) > 0:
                num_up += 1
            else:
                num_down += 1
        isoform = isoform_results[0]
        classification = isoform_classification[isoform]
        categories.append(classification)

    # Append to table data
    table_data.append([
        cur_gene,
        illumina_fold,  # Ensure numbers are formatted properly
        illumina_p,
        num_isoforms,
        num_up,
        num_down,
        "|".join(categories)
    ])

# Create a DataFrame
df = pd.DataFrame(table_data, columns=header)

# # Format numeric columns for better readability
# df["log2FoldChange"] = df["log2FoldChange"].map(lambda x: f"{x:.3f}")
# df["padj"] = df["padj"].map(lambda x: f"{x:.3e}")  # Scientific notation for small p-values

# Save to CSV (for sharing)
df.to_csv("tip60_table.csv", index=False)

# Print a nicely formatted table manually
print("\t".join(header))  # Print header row
for row in table_data:
    print("\t".join(map(str, row)))  # Print each row with tab separation


Gene	log2FoldChange	padj	num_isoforms	significant_up	significant_down	categories
KAT5	-0.410763045	0.000225465	12	0	2	alternative_5end|reference_match|reference_match|3prime_fragment|3prime_fragment|alternative_3end|alternative_5end|alternative_5end|combination_of_known_junctions|alternative_3end5end|3prime_fragment|reference_match
EP400	-0.280616191	0.038109071	1	0	0	combination_of_known_junctions
TRRAP	-0.138325654	0.414363477	6	0	0	internal_fragment|internal_fragment|internal_fragment|internal_fragment|internal_fragment|internal_fragment
BRD8	0.157954396	0.35760197	15	1	0	combination_of_known_junctions|combination_of_known_junctions|3prime_fragment|3prime_fragment|reference_match|combination_of_known_junctions|reference_match|reference_match|combination_of_known_junctions|3prime_fragment|3prime_fragment|combination_of_known_junctions|3prime_fragment|combination_of_known_junctions|reference_match
EPC1	0.169092142	0.139968173	6	1	0	reference_match|reference_match|alternative_3end|5pri

In [11]:
import pandas as pd
from collections import Counter

# Define table structure
table_data = []

# Define broad isoform categories
category_mapping = {
    "reference_match": "Canonical isoform",
    "alternative_3end": "Alternative splicing",
    "alternative_5end": "Alternative splicing",
    "intron_retention": "Alternative splicing",
    "at_least_one_novel_splicesite": "Novel isoform",
    "combination_of_known_splicesites": "Novel isoform",
    "3prime_fragment": "Truncated transcript",
    "5prime_fragment": "Truncated transcript"
}

# Function to determine the dominant isoform category for a gene
def summarize_categories(categories):
    mapped_categories = [category_mapping.get(cat, "Other") for cat in categories]
    
    unique_cats = set(mapped_categories)
    
    return '|'.join(unique_cats)
#     most_common = Counter(mapped_categories).most_common(1)
#     return most_common[0][0] if most_common else "Unknown"

# Print header
header = ["Gene", "log2FoldChange", "padj", "num_isoforms", "significant_up", "significant_down", "categories", "summary_category"]

for cur_gene in genes:
    illumina_fold = illumina_lookup[cur_gene][2]
    illumina_p = illumina_lookup[cur_gene][5]

    num_isoforms = len(pacbio_lookup[cur_gene])
    categories = []
    num_up = 0
    num_down = 0

    for isoform_results in pacbio_lookup[cur_gene]:
        if float(isoform_results[5]) <= 0.05:  # Adjust significance threshold if needed
            if float(isoform_results[2]) > 0:
                num_up += 1
            else:
                num_down += 1
        isoform = isoform_results[0].replace('"', '')
        classification = isoform_classification[isoform]
        categories.append(classification)

    # Determine dominant isoform category
    summary_category = summarize_categories(categories)

    # Append to table data
    table_data.append([
        cur_gene,
        illumina_fold,  # Ensure numbers are formatted properly
        illumina_p,
        num_isoforms,
        num_up,
        num_down,
        "|".join(categories),
        summary_category
    ])

# Create a DataFrame
df = pd.DataFrame(table_data, columns=header)

# Save to CSV
df.to_csv("tip60_table_summarized.csv", index=False)

# Print a nicely formatted table manually
print("\t".join(header))  # Print header row
for row in table_data:
    print("\t".join(map(str, row)))  # Print each row with tab separation


Gene	log2FoldChange	padj	num_isoforms	significant_up	significant_down	categories	summary_category
KAT5	-0.410763045	0.000225465	13	0	2	reference_match|reference_match|alternative_5end|alternative_5end|alternative_3end|reference_match|combination_of_known_junctions|alternative_3end5end|alternative_5end|3prime_fragment|3prime_fragment|3prime_fragment|3prime_fragment	Canonical isoform|Truncated transcript|Other|Alternative splicing
EP400	-0.280616191	0.038109071	1	0	0	combination_of_known_junctions	Other
TRRAP	-0.138325654	0.414363477	4	0	0	internal_fragment|internal_fragment|internal_fragment|internal_fragment	Other
BRD8	0.157954396	0.35760197	15	0	0	reference_match|combination_of_known_junctions|combination_of_known_junctions|reference_match|combination_of_known_junctions|reference_match|combination_of_known_junctions|combination_of_known_junctions|combination_of_known_junctions|3prime_fragment|3prime_fragment|3prime_fragment|3prime_fragment|reference_match|3prime_fragment	Canonical iso

In [13]:
import pandas as pd
from collections import Counter

# Define table structure
table_data = []

# Define broad isoform categories
category_mapping = {
    "reference_match": "Canonical isoform",
    "alternative_3end": "Alternative splicing",
    "alternative_5end": "Alternative splicing",
    "intron_retention": "Alternative splicing",
    "at_least_one_novel_splicesite": "Novel isoform",
    "combination_of_known_splicesites": "Novel isoform",
    "3prime_fragment": "Truncated transcript",
    "5prime_fragment": "Truncated transcript"
}

# Function to determine the dominant isoform category for a gene
def summarize_categories(categories):
    mapped_categories = [category_mapping.get(cat, "Other") for cat in categories]
    
    unique_cats = set(mapped_categories)
    
    return '|'.join(unique_cats)

# Print header
header = ["Gene", "log2FoldChange", "padj", "num_isoforms", "significant_up", "significant_down", "categories", "summary_category"]

for cur_gene in genes:
    illumina_fold = illumina_lookup[cur_gene][2]
    illumina_p = illumina_lookup[cur_gene][5]

    num_isoforms = len(pacbio_lookup[cur_gene])
    categories = []
    num_up = 0
    num_down = 0

    for isoform_results in pacbio_lookup[cur_gene]:
        if float(isoform_results[5]) <= 0.05:  # Adjust significance threshold if needed
            if float(isoform_results[2]) > 0:
                num_up += 1
            else:
                num_down += 1
        isoform = isoform_results[0].replace('"', '')
        classification = isoform_classification[isoform]
        categories.append(classification)

    # Add a star to the gene name if padj < 0.05
    gene_name = cur_gene
    if float(illumina_p) < 0.05:
        gene_name = f"{cur_gene}*"

    # Determine dominant isoform category
    summary_category = summarize_categories(categories)

    # Append to table data
    table_data.append([gene_name,
        illumina_fold,  # Ensure numbers are formatted properly
        illumina_p,
        num_isoforms,
        num_up,
        num_down,
        "|".join(categories),
        summary_category
    ])

# Create a DataFrame
df = pd.DataFrame(table_data, columns=header)

# Save to CSV
df.to_csv("tip60_table_summarized.csv", index=False)

# Display the table without the row numbers
df


,Gene,log2FoldChange,padj,num_isoforms,significant_up,significant_down,categories,summary_category
0,KAT5*,-0.410763045,0.000225465,13,0,2,reference_match|reference_match|alternative_5e...,Canonical isoform|Truncated transcript|Other|A...
1,EP400*,-0.280616191,0.038109071,1,0,0,combination_of_known_junctions,Other
2,TRRAP,-0.138325654,0.414363477,4,0,0,internal_fragment|internal_fragment|internal_f...,Other
3,BRD8,0.157954396,0.35760197,15,0,0,reference_match|combination_of_known_junctions...,Canonical isoform|Truncated transcript|Other
4,EPC1,0.169092142,0.139968173,6,1,0,alternative_3end|5prime_fragment|5prime_fragme...,Canonical isoform|Truncated transcript|Alterna...
5,EPC2,0.064374159,0.701234555,4,0,0,reference_match|3prime_fragment|3prime_fragmen...,Canonical isoform|Truncated transcript
6,MBTD1,-0.181013499,0.07057304,2,0,0,5prime_fragment|at_least_one_novel_splicesite,Truncated transcript|Novel isoform
7,DMAP1,-0.145453953,0.279852815,13,0,2,reference_match|combination_of_known_junctions...,Truncated transcript|Alternative splicing|Othe...
8,ING3,0.024888729,0.923187566,8,1,1,alternative_3end|alternative_3end|reference_ma...,Canonical isoform|Alternative splicing
9,VPS72,-0.10319127,0.650861628,13,2,1,alternative_3end|alternative_3end|at_least_one...,Truncated transcript|Other|Alternative splicin...
